## Auto Deep Learning: Image Classification

PACE-ML Auto DL will run and evaluate several deep learning models of various architectures  on the user provided data. It will identify the best performing deep learning model architecture  on the basis of validation metric for image classification. This will reduce the  time and effort for the model building task for a data scientist. This becomes an important component of the ML-OPS framework for data science from automation perspective.


### Contents

1. [Set up the environment](#Set-up-the-environment)
1. [Usage Instructions](#Usage-Instructions)
1. [Upload the data for training](#Upload-the-data-for-training)
1. [Run Training Job](#Run-Training-Job)
1. [Live Inference Endpoint](#Live Inference)
1. [Batch Transform Job](#Batch-Transform-Job)
1. [Output Interpretation](#Output-Interpretation)



<img src="images/Flow_diagram.JPG">

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

### Input format
#### Input:
•	This algorithm takes a zip file as an input. This zip file to be uploaded for training the model should follow the following file structure.<br><br>

Sample.zip

Sample.zip
|----class 1
|----|----img1.png
|----class 2
|----|----img1.png
|----…
|----class n
|----|----img1.png
|----dict.json



•  This zip file to be uploaded for testing the trained model should follow the following file structure.<br><br>

Sample.zip
|----img1.png
|----img2.png
|----…
|----img.png

•	Each class folder can have multiple images relevant to that class. The image file format supported are ‘png’ and ‘jpg’

## Set up the environment
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [25]:
# S3 prefix
prefix = 'Image-classifier'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session
The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [26]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training
When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using classification dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [28]:
data_location= 's3://aws-marketplace-mphasis-assets/image_classifier/sample_data.zip'
data_location

's3://aws-marketplace-mphasis-assets/image_classifier/sample_data.zip'

## Create an estimator and fit the model
In order to use SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:
- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above

Then we use fit() on the estimator to train against the data that we uploaded above.

In [35]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/image-dl-2:latest'.format(account, region)
tree = sage.estimator.Estimator(image,
                       role, 3, 'ml.c4.2xlarge',
                      output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)


2021-03-22 13:11:28 Starting - Starting the training job...
2021-03-22 13:11:51 Starting - Launching requested ML instancesProfilerReport-1616418687: InProgress
......
2021-03-22 13:12:52 Starting - Preparing the instances for training......
2021-03-22 13:13:52 Downloading - Downloading input data
2021-03-22 13:13:52 Training - Downloading the training image.........
2021-03-22 13:15:18 Training - Training image download completed. Training in progress.2021-03-22 13:15:18.530849: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-03-22 13:15:18.530932: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-03-22 13:15:17.884315: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: li

## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.


In [36]:
training_job_name = tree.latest_training_job.name
attached_tree = sage.estimator.Estimator.attach(training_job_name)


2021-03-22 13:16:52 Starting - Preparing the instances for training
2021-03-22 13:16:52 Downloading - Downloading input data
2021-03-22 13:16:52 Training - Training image download completed. Training in progress.
2021-03-22 13:16:52 Uploading - Uploading generated training model
2021-03-22 13:16:52 Completed - Training job completed



### Deploy the model
Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [37]:

from sagemaker.predictor import csv_serializer
predictor = attached_tree.deploy(4, 'ml.m4.xlarge', serializer=csv_serializer,endpoint_name='image-classifier')

---------------!

## Choose some data and use it for a prediction


In [43]:
test_data  = 's3://mphasis-marketplace/image_classifier/input/sample_test.zip'



### Output

Output files contains column predicted Group, which has the predicted class

In [46]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path)

In [47]:
transformer.transform(test_data, content_type='application/zip')
transformer.wait()
print("Batch Transform output saved to " + transformer.output_path)

...............................Starting the inference server with 4 workers.
[2021-03-22 13:36:02 +0000] [9] [INFO] Starting gunicorn 20.0.4
[2021-03-22 13:36:02 +0000] [9] [INFO] Listening at: unix:/tmp/gunicorn.sock (9)
[2021-03-22 13:36:02 +0000] [9] [INFO] Using worker: gevent
[2021-03-22 13:36:02 +0000] [12] [INFO] Booting worker with pid: 12
[2021-03-22 13:36:02 +0000] [13] [INFO] Booting worker with pid: 13
[2021-03-22 13:36:02 +0000] [17] [INFO] Booting worker with pid: 17
[2021-03-22 13:36:02 +0000] [21] [INFO] Booting worker with pid: 21
2021-03-22 13:36:02.685624: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-03-22 13:36:02.686010: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-03-22 13:36:02.765413: W tensorflow/stream_executor/p

#### Inspect the Batch Transform Output in S3